In [ ]:
from datasets import load_dataset
import matplotlib.pyplot as plt
from transformers import EfficientNetModel
from torchvision import datasets, transforms
from torchvision.transforms.functional import pil_to_tensor
import torch
import numpy as np

/home/kenfus/miniconda3/envs/flaresense-v2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
ds = load_dataset('i4ds/radio-sunburst-ecallisto')

In [ ]:
def to_torch_tensor(example):
    # Convert the PIL image to a NumPy array for checking
    np_image = np.array(example['image'])

    # Check if any value is above 255
    if np.any(np_image > 255):
        print("Found value above 255 in example:", example)

    # Check if the image is not uint8
    if np_image.dtype != np.uint8:
        print("Image is not of type uint8 in example:", example)

    # Convert the PIL image to a tensor
    example['image'] = pil_to_tensor(example['image'])
    return example


def scale(example, max_value=255):
    example['image'] = torch.div(example['image'], max_value)
    return example

In [ ]:
ds_tensor = ds.map(to_torch_tensor)

Map:   1%|▏         | 954/72123 [00:17<01:02, 1142.29 examples/s]/home/kenfus/miniconda3/envs/flaresense-v2/lib/python3.12/site-packages/datasets/features/image.py:332: UserWarning: Downcasting array dtype int64 to uint8 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")
Map: 100%|██████████| 9010/9010 [00:47<00:00, 188.45 examples/s]


In [ ]:
ds_tensor = ds_tensor.map(scale)

In [ ]:
mean = torch.mean(torch.stack(ds_tensor['train']['image']))
std = torch.std(torch.stack(ds_tensor['train']['image']))

NameError: name 'torch' is not defined

In [ ]:
from torchvision.transforms import Resize, Compose, Normalize

In [ ]:
normalize = Normalize(mean=mean, std=std)
size = (224, 244)

In [ ]:
_transforms = Compose([Resize(size), ToTensor(), normalize])

In [ ]:
model = EfficientNetModel.from_pretrained('google/efficientnet-b0')